In [1]:
%pylab inline
from numpy.lib.format import open_memmap
import pandas as pd
import weighted

Populating the interactive namespace from numpy and matplotlib


In [2]:
def values_states(desired_state):
    """
    Reads in census data csv. Returns only data for desired state.
    """
    chunk_1 = pd.DataFrame()
    reader_a = pd.read_csv('ss16pusa.csv',memory_map=True,chunksize=80000)
    for a in reader_a:
        chunk_1 = chunk_1.append(a[a['ST']==desired_state])
        if max(a['ST'])>desired_state:
            break
    reader_b = pd.read_csv('ss16pusb.csv',memory_map=True,chunksize=80000)
    for a in reader_b:
        chunk_1 = chunk_1.append(a[a['ST']==desired_state])
        if max(a['ST'])>desired_state:
            break
    return chunk_1

In [3]:
def racially_biased_salaries(desired_state,len_chosen = 10000,check_file = True,filename = 'states.race.income.dat'):
    """
    Returns the weighted median income of white and black respondants after controlling for sex, age, and education.
    If check_file, execute only if state isn't already done.
    
    """
    if check_file:
        states_done = np.loadtxt(filename,skiprows=1)
        if len(states_done) == 0:
            pass
        elif len(states_done.shape)==1:
            if states_done[0] == desired_state:
                return 'Finished'
            else:
                pass
        elif desired_state in states_done.T[0]:
            return 'Finished'
    chunk_1 = values_states(desired_state)
    some_hs = chunk_1[(chunk_1['SCHL']>11)]
    bl_some_hs = chunk_1[(chunk_1['SCHL']>11)&(chunk_1['RAC1P']==2)&(chunk_1['AGEP']>18)&(chunk_1['PINCP']>0)]
    wh_some_hs = chunk_1[(chunk_1['SCHL']>11)&(chunk_1['RAC1P']==1)&(chunk_1['AGEP']>18)&(chunk_1['PINCP']>0)]
    rand_targets = np.array([np.sum(wh_some_hs['PWGTP'][:i+1]) for i in range(len(wh_some_hs['PWGTP'])+1)])
    chosen = []
    acts = np.random.randint(0,rand_targets[-1],len_chosen)
    for act in acts:
        finder = rand_targets<=act
        try:
            chosen.append(list(finder).index(False))
        except IndexError:
            chosen.append(-1)
    chosen = wh_some_hs.index[chosen]
    wh_final = []
    bl_final = []
    for i in chosen:
        matches = bl_some_hs[(bl_some_hs['AGEP'] == wh_some_hs.loc[i]['AGEP'])&(bl_some_hs['SCHL'] == wh_some_hs.loc[i]['SCHL'])]
        if len(matches) == 0:
            continue
        weighted_matches = np.array([np.sum(matches['PWGTP'][:j+1]) for j in range(len(matches))])
        act_ch = np.random.randint(0,weighted_matches[-1])  
        wh_final.append(i)
        bl_final.append(matches.index[list(act_ch <=weighted_matches).index(True)])
    wh_analyze = wh_some_hs.loc[wh_final]
    bl_analyze = bl_some_hs.loc[bl_final]
    wh_median = np.median(wh_analyze['PINCP'])
    bl_median = np.median(bl_analyze['PINCP'])
    return [wh_median,bl_median]

This cell makes the list of all state codes. Presumably the skipped values are US territories or similar

In [ ]:
states = [1,2,4,5,6,8,9,10,11,12,13,15,16,17,18,19,20,21,22,23,24]
states.extend(np.arange(25,43))
states.extend(np.arange(44,52))
states.extend(np.arange(53,57))


Running this block will write the median white/black salaries to "states.race.income.dat". From there we can plot them however we choose.

In [ ]:
w = []
b = []
for i in states:
    data = racially_biased_salaries(i,14000)
    if data == 'Finished':
        continue
    w1,b1 = data
    f=open("states.race.income.dat", "a+")
    f.write('{0} {1} {2}\n'.format(i,w1,b1))
    f.close()